In [22]:
import spacy
import mysql.connector
import json

In [23]:
# Connect to the WAMP MySQL database
db = mysql.connector.connect(
    host="localhost",  # Assuming WAMP server is running locally
    user="root",
    password="",
    database="badrpayroll"
)

cursor = db.cursor(dictionary=True)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Minimum wage in QAR
MINIMUM_WAGE = 1000

def check_minimum_wage():
    query = """
    SELECT e.EmployeeID, e.FirstName, e.LastName, SUM(ew.WagePaid) AS MonthlySalary
    FROM employee e
    JOIN employee_wages ew ON e.EmployeeID = ew.EmployeeID
    WHERE ew.Date BETWEEN DATE_FORMAT(CURRENT_DATE() ,'%Y-%m-01') AND LAST_DAY(CURRENT_DATE())
    GROUP BY e.EmployeeID
    HAVING MonthlySalary < %s
    """
    cursor.execute(query, (MINIMUM_WAGE,))
    results = cursor.fetchall()
    return results

def generate_summary(non_compliant_employees):
    summary_prompt = "Generate a report for the following employees who are paid below the minimum wage:\n"
    for employee in non_compliant_employees:
        summary_prompt += f"Employee: {employee['FirstName']} {employee['LastName']}, Monthly Salary: {employee['MonthlySalary']} QAR\n"
    
    doc = nlp(summary_prompt)
    summary = " ".join([sent.text for sent in doc.sents])
    return summary

# Fetch non-compliant employees
non_compliant_employees = check_minimum_wage()

# Generate summary
if non_compliant_employees:
    summary = generate_summary(non_compliant_employees)
    output = {
        "status": "Non-compliance found",
        "summary": summary,
        "details": non_compliant_employees
    }
else:
    output = {
        "status": "All employees are paid at or above the minimum wage.",
        "summary": "",
        "details": []
    }

# Ensure only JSON is printed
print(json.dumps(output, indent=4))

# Close the database connection
cursor.close()
db.close()



{
    "status": "Non-compliance found",
    "summary": "Generate a report for the following employees who are paid below the minimum wage:\n Employee: John Doe, Monthly Salary: 500.0 QAR\n",
    "details": [
        {
            "EmployeeID": 1,
            "FirstName": "John",
            "LastName": "Doe",
            "MonthlySalary": 500.0
        }
    ]
}
